In [27]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [28]:
import os
import urllib.parse

In [29]:
import requests
import brotli
from bs4 import BeautifulSoup

In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [31]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [32]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

In [33]:
logfile_name = "_seleniumtest_log.txt"

# import sk & tw politician test data

In [34]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [35]:
sktest.shape

(9960, 24)

In [36]:
# sktest["URL_NamuWiki"]=np.nan

In [39]:
sktest[3000:3100]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
3000,문병원,文炳元,19570507,남,57,광역의원비례대표선거,20140604,Party List Legislator,울산광역시,NaN,...,등록,1,NaN,NaN,NaN,사회복지,동국대 사회과학대학원 졸업(사회복지학),(현)밝은미래복지재단 이사,(전)제18대 대선 새누리당 울산선대위 선거대책위원장,NaN
3001,문병호,文炳浩,19591227,남,56,국회의원선거,20160413,Legislator,인천광역시,부평구,...,등록,0,NaN,NaN,NaN,국회의원,서울대학교 법과대학 법학과 졸업,(현)변호사(민변),(현)국민의당 인천광역시당위원장,NaN
3002,문병화,文炳華,19620404,남,52,구·시·군의회의원선거,20140604,Legislator,경기도,하남시,...,등록,0,NaN,NaN,NaN,회사원,전신서해공업전문대학(현 호원대학교) 토목과 졸업 (2년),(현) 홍운건설 이사,(현) 하남시문화원 이사,NaN
3003,문병훈,文柄勳,19790315,남,35,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,1,8385.0,34.17,NaN,연구원,한양대학교 도시대학원 석사과정 졸업(도시학석사),(전)경기개발연구원 도시지역계획연구부 연구원,(현)한국산지보전협회 선임연구원,NaN
3004,문보현,文普玄,19650519,남,50,국회의원선거,20160413,Legislator,전라남도,목포시,...,등록,0,NaN,NaN,NaN,정당인,나고야대학대학원 법학박사과정수료(2000.4.1~2006.3.31.),(전)목포대학교 총학생회장,(현)정의당 정책연구위원,NaN
3005,문복란,文福蘭,19560601,여,58,구·시·군의회의원선거,20140604,Legislator,서울특별시,성동구,...,등록,1,6902.0,19.15,NaN,정당인,한국방송통신대학교 교육학과 졸업,(현) 새정치민주연합 서울특별시당 성범죄예방특별위원회 부위원장,(현) 대한적십자사 서울특별시지사 성동구후원회 사무국장,NaN
3006,문봉도,文奉道,19650505,남,49,구·시·군의회의원선거,20140604,Legislator,경상남도,의령군,...,등록,1,1293.0,33.75,NaN,의령군의회의원,한국방송통신대학교 행정학과 졸업,(현)제6대 의령군의회의원,(현)제6대 의령군의회 상·하반기 산업건설위원장,NaN
3007,문봉선,文峰仙,19620310,여,52,구·시·군의회의원선거,20140604,Legislator,경기도,과천시,...,등록,1,4713.0,26.36,NaN,정당인,동국대학교 문화예술대학원 졸업(문학 석사),(전)새누리당 과천의왕 당협 문화예술분과위원장,(현)한국문인협회 과천지부 상임 부회장,NaN
3008,문상모,文鋿模,19690202,남,45,시·도의회의원선거,20140604,Legislator,서울특별시,노원구,...,등록,1,22270.0,53.83,NaN,서울특별시의회 의원,광운대학교 상담복지정책대학원 사회복지학과 재학,(현)서울특별시의회 문화체육관광위원회 위원,(현)2018 평창동계올림픽지원 및 서울시스포츠 활성화를 위한 특별위원장,NaN
3009,문상선,文相宣,19670404,남,47,구·시·군의회의원선거,20140604,Legislator,경기도,의정부시,...,등록,0,NaN,NaN,NaN,정당인,한국방송통신대학교 경영학과 1학년 제적,(전) 민주당 국회의원 김진표 정책특보,(현) 의정부시 청소년지도협의회 사무국장,NaN


In [40]:
# percent duplicates

1 - (sktest["Name_Native"].drop_duplicates().shape[0] / sktest.shape[0])

0.12550200803212852

# scrape namuwiki with selenium

In [13]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

In [41]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url - encode non-ASCII search terms as ASCII
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""

        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                #sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,url,"\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")

            else:

                # update HasNamuWiki content field in dataframe
                # sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,"False","\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")
            
        except KeyboardInterrupt:
        
            print("Keyboard Interrupt")
            
            break

        except Exception as e:

            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            print(e)





2506 김하용 https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%9A%A9 Number of <p> elements: 4 Has Content
2507 김하중 https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%A4%91 Number of <p> elements: 4 Has Content
2508 김학관 https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B4%80 Number of <p> elements: 4 Has Content
2509 김학구 https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B5%AC Number of <p> elements: 5 Empty
2510 김학규 https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B7%9C Number of <p> elements: 4 Has Content
2511 김학기 https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B8%B0 Number of <p> elements: 4 Has Content
2512 김학두 https://namu.wiki/w/%EA%B9%80%ED%95%99%EB%91%90 Number of <p> elements: 5 Empty
2513 김학만 https://namu.wiki/w/%EA%B9%80%ED%95%99%EB%A7%8C Number of <p> elements: 5 Empty
2514 김학성 https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%84%B1 Number of <p> elements: 4 Has Content
2515 김학수 https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%88%98 Number of <p> elements: 4 Has Content
2516 김학영 https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%98%81 Number of 

In [61]:
sktest[sktest.Name_Native=="강민국"]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
60,강민국,姜旻局,19710303,남,43,시·도의회의원선거,20140604,Legislator,경상남도,진주시,...,등록,1,28248.0,75.31,NaN,정당인,경남대학교 대학원 졸업(법학박사),(전)경상남도 도지사 비서실장,(전)경상남도 정무보좌역,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD


In [ ]:
# things to check

# 0. drop duplicates. some duplicate pages are different (at least size differs). why?  

# 0.1 drop duplicates (by=[Name_Native, Name_Chinese, Birthdate]) in TW dataset, before scraping

# 1. saved links are valid (no false positives); False links actually don't exist (no false negatives)

# 2. disambiguation? what about names that link to more than one person?
#
#       - disambiguation links are usually included in the page
#
#       - redirect from the page of a more popular person
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#               정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)
#
#       - redirects from a page listing links to multiple people having the same name
#               https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84

# update sktest with selenium log from latest run

In [42]:
log_columns = ["index","Name_Native","url","dropme"]
log = pd.read_csv(fileout_path+logfile_name,header=None,encoding="utf-8",names=log_columns)
log.drop(columns=["dropme"],inplace=True)
log

,index,Name_Native,url
0,2506,김하용,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%9A%A9
1,2507,김하중,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%A4%91
2,2508,김학관,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B4%80
3,2509,김학구,False
4,2510,김학규,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B7%9C
5,2511,김학기,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B8%B0
6,2512,김학두,False
7,2513,김학만,False
8,2514,김학성,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%84%B1
9,2515,김학수,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%88%98


In [43]:
sktest.shape

(9960, 24)

In [44]:
sktest_updated = sktest.merge(log,how="left",on="Name_Native")
sktest_updated.shape

(9966, 26)

In [45]:
# update URL_NamuWiki with log results
sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"URL_NamuWiki"] = sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"url"]

# confirm that changes have been made
sktest_updated.loc[sktest_updated.url.notnull(),["URL_NamuWiki","url"]]

,URL_NamuWiki,url
2506,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%9A%A9,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%9A%A9
2507,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%A4%91,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%A4%91
2508,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B4%80,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B4%80
2509,False,False
2510,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B7%9C,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B7%9C
2511,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B8%B0,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B8%B0
2512,False,False
2513,False,False
2514,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%84%B1,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%84%B1
2515,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%88%98,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%88%98


In [46]:
# change "False" strings to boolean False
sktest_updated.loc[sktest_updated.URL_NamuWiki=="False","URL_NamuWiki"]=False

In [48]:
sktest_updated[sktest_updated["index"].notnull()]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki,index,url
2506,김하용,金夏龍,19501105,남,63,구·시·군의회의원선거,20140604,Legislator,경상남도,창원시진해구,...,4203.0,25.28,NaN,수산업,부경대학교 대학원 수해양인적자원개발학과 석사과정 휴학중,(전)통합창원시의회 초대 시의원,(전)의창수산업협동조합장 3대역임,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%9A%A9,2506.0,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%9A%A9
2507,김하중,金夏中,19600620,남,55,국회의원선거,20160413,Legislator,광주광역시,서구,...,NaN,NaN,NaN,전남대학교 로스쿨 교수,고려대학교 졸업(법학박사),(전)서울중앙지검 부장검사,(현)전남대학교 로스쿨 교수,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%A4%91,2507.0,https://namu.wiki/w/%EA%B9%80%ED%95%98%EC%A4%91
2508,김학관,金學灌,19550915,남,58,구·시·군의 장선거,20140604,Chief,전라북도,임실군,...,NaN,NaN,NaN,정치인,전주대학교 경영대학원 석사학위과정 1학기 휴학,(전)5대 임실군의회 전·후반기 의장,(현)(사)임실군4-H본부장,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B4%80,2508.0,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B4%80
2509,김학구,金學球,19640312,남,50,구·시·군의회의원선거,20140604,Legislator,경상북도,문경시,...,NaN,NaN,NaN,건설기계 굴삭기 운전,대유공업전문대학 졸업(건축과),(전)전국건설기계연합회문경시지회 회장,(현)체불없는 문경시건설연합회 회장,False,2509.0,False
2510,김학규,金學奎,19471214,남,66,구·시·군의 장선거,20140604,Chief,경기도,용인시처인구,...,NaN,NaN,NaN,선출직 공무원(용인시장),단국대학교 행정대학원 수료(행정학 박사과정),(전)경기도의회 의원,(현)용인시장,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B7%9C,2510.0,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B7%9C
2511,김학기,金學起,19600324,남,54,구·시·군의 장선거,20140604,Chief,대구광역시,달서구,...,NaN,NaN,NaN,정당인,계명대학교 중국학과 3년 제적,(전)노무현대통령 비서실 행정관,(전)문재인 대통령후보 대구선거대책본부장,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B8%B0,2511.0,https://namu.wiki/w/%EA%B9%80%ED%95%99%EA%B8%B0
2512,김학두,金學杜,19640520,남,50,시·도의회의원선거,20140604,Legislator,서울특별시,동대문구,...,NaN,NaN,NaN,정당인,유한공업전문대학 공업디자인과 졸업 (현 유한대학교),(전) 제6대 동대문구의회의원,(전) 박근혜 대통령후보 지역책임본부장,False,2512.0,False
2513,김학만,金學萬,19571204,남,56,시·도의회의원선거,20140604,Legislator,경기도,평택시,...,NaN,NaN,NaN,자산학원 이사장,아주대학교 교육대학원 (교육학석사),(전)제6대 경기도 도의원,(현)자산학원이사장,False,2513.0,False
2514,김학성,金學性,19571224,남,56,구·시·군의회의원선거,20140604,Legislator,전라북도,순창군,...,NaN,NaN,NaN,농업,미기재,(전)동계중·고등학교 운영위원장,(현)동계초등학교 총동문회 사무처장,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%84%B1,2514.0,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%84%B1
2515,김학수,金學洙,19660607,남,47,구·시·군의회의원선거,20140604,Legislator,강원도,원주시,...,7468.0,29.00,NaN,원주시의원,강원대학교 공과대학 발효공학과 졸업,(전)사단법인 한국음식업중앙회 원주시지부장,(현)원주시의회 의원,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%88%98,2515.0,https://namu.wiki/w/%EA%B9%80%ED%95%99%EC%88%98


In [50]:
# drop log column (url)
sktest_updated.drop(columns=["index","url"],inplace=True)

KeyError: "['index' 'url'] not found in axis"

In [51]:
# drop duplicate rows
sktest_updated.drop_duplicates().shape

(9960, 24)

In [52]:
# drop duplicate politicians
sktest_updated.sort_values(["Name_Native","Name_Chinese","Birthdate","ElectionDate"],inplace=True)
sktest_updated.drop_duplicates(["Name_Native","Name_Chinese","Birthdate"],keep="last",inplace=True)

In [54]:
sktest_updated.shape

(9960, 24)

In [55]:
# refresh and save updated sktest
sktest = sktest_updated
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [56]:
# delete log file
os.remove(fileout_path+logfile_name)

In [33]:
# DEPRECATED - assign URL_NamuWiki link based on files already scraped - I needed this before I created a log file

# import os
# filelist = os.listdir(fileout_path)
# test = sktest
# url_stem = "https://namu.wiki"
# test[100:200]
# scraped = [f.split("_")[2].split(".")[0].strip() for f in filelist]
# for s in scraped:
#     url = url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8"))
#     # print(url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8")))
#     test.loc[test.Name_Native==s,"URL_NamuWiki"]=url
# test[100:200]